<div style="text-align: center;">
<h1>Kernel Density Estimation</h1>
<h3>By Jit Mondal & Sayan Dewanjee</h3>
</div>

# Install required packages

In [44]:
!pip install numpy
!pip install plotly
!pip install scipy
!pip install matplotlib
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# importing module
import numpy as np
import matplotlib.pyplot as plt
import plotly as plotly
import plotly.io as plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
from scipy.stats import chi2

In [6]:
# class for random sample
class random_sample:
  def __init__(self,n):
    self.n=n

  def gaussian(self,mu=0,sigma=1):
    return np.random.normal(mu,sigma,self.n)

  def cauchy(self ,mu=0, gamma=1):
    return np.random.standard_cauchy(self.n)*gamma+mu

  def exp(self,alpha=1):
    return np.random.exponential(alpha,self.n)
  def pareto(self,alpha=1,x_m=1):
    return x_m * (np.random.pareto(alpha, self.n) + 1)
  def logistic(self,mu=0,scale=1):
    return np.random.logistic(mu,scale,self.n)
  def gumbel(self,mu=0,sigma=1):
    return np.random.gumbel(mu,sigma,self.n)
  def tri(self,left=-1,mode=0,right=1):
    return np.random.triangular(left, mode, right, self.n)
  def unif(self,low=0,high=1):
    return np.random.uniform(low,high,self.n)
  def chi2(self,df=1):
    return np.random.chisquare(df,self.n)

In [7]:

# class of kernels
class density:
  def __init__(self,x):
    self.x = x

  def biweight_den(self):
    if abs(self.x) < 1:
      return (15/16)*(1-self.x**2)**2
    else:
      return 0

  def rect_den(self):
      x=self.x
      x = np.asarray(x)
      pdf_values = np.zeros_like(x, dtype=float)
      mask = abs(x) < 1
      pdf_values[mask] = 0.5
      return pdf_values


  def tri_den(self):
      x=self.x
      x = np.asarray(x)
      pdf_values = np.zeros_like(x, dtype=float)
      mask = abs(x) < 1
      pdf_values[mask] = 1-abs(x[mask])
      return pdf_values

  def gaussian_den(self):
    return np.exp(-((self.x)**2)/2)/(np.sqrt(2*np.pi))
  def chi2_den(self,df):
    return chi2.pdf(self.x, df=df)

  def epanechnikov_den(self):
      x=self.x
      x = np.asarray(x)
      pdf_values = np.zeros_like(x, dtype=float)
      mask = abs(x) < np.sqrt(5)
      pdf_values[mask] = ((3/4)*(1 - (x[mask]**2)/5))/np.sqrt(5)
      return pdf_values


  def cauchy_den(self):
    return 1/(np.pi*(1+((self.x)**2)))


  def exp_den(self):
    x=self.x
    x = np.asarray(x)
    pdf_values = np.zeros_like(x, dtype=float)
    mask = x > 0
    pdf_values[mask] = np.exp(-x[mask])
    return pdf_values




  def pareto_den(self, alpha, xm):

      x=self.x
      x = np.asarray(x)
      pdf_values = np.zeros_like(x, dtype=float)
      mask = x >= xm
      pdf_values[mask] = (alpha * (xm ** alpha)) / (x[mask] ** (alpha + 1))
      return pdf_values

  def gumbel_den(self, mu=0, sigma=1):

    x=self.x
    z = (x - mu) / sigma
    return (1 / sigma) * np.exp(-z - np.exp(-z))

  def unif(self,a=0,b=1):
    x=self.x
    x = np.asarray(x)
    pdf_values = np.zeros_like(x, dtype=float)
    mask = (x >= a) & (x <= b)
    pdf_values[mask] = 1/(b-a)
    return pdf_values
  def special_den(self):
    x=self.x
    x = np.asarray(x)
    pdf_values = np.zeros_like(x, dtype=float)
    mask = abs(x) < 1
    pdf_values[mask] = ((3/8)*(3 - 5*(x[mask]**2)))
    return pdf_values


In [8]:
# calling the appropiate density function
def den(x,pdf):
  pdf=pdf.lower()
  if "gaussian" in pdf:
    return density(x).gaussian_den()

  elif "cauchy" in pdf:
    return density(x).cauchy_den()

  elif "epanechnikov" in pdf:
    return density(x).epanechnikov_den()

  elif "exp" in pdf:
    return density(x).exp_den()

  elif "rect" in pdf:
    return density(x).rect_den()

  elif "tri" in pdf:
    return density(x).tri_den()

  elif "biweight" in pdf:
    return density(x).biweight_den()
  elif "pareto" in pdf:
    return density(x).pareto_den(alpha=eval(pdf.split("_")[1]),xm=eval(pdf.split("_")[2]))
  elif "gumbel" in pdf:
    return density(x).gumbel_den(mu=eval(pdf.split("_")[1]),sigma=eval(pdf.split("_")[2]))
  elif "unif" in pdf:
    return density(x).unif(a=eval(pdf.split("_")[1]),b=eval(pdf.split("_")[2]))
  elif "chi" in pdf:
    return density(x).chi2_den(df=eval(pdf.split("_")[1]))
  elif "special" in pdf:
    return density(x).special_den()
  else:
    l=["gaussian","cauchy","epanechnikov","exp","rect","tri","biweight","pareto","gumbel"]
    return f"Please choose the pdf from one of the following: \n{l}"

In [9]:
#code for f_n(x)
def f_n(x,h,X,kernel):
  n=len(X)
  sum=0
  kernel=kernel.lower()
  try:
    for i in range(n):
      sum+=den(((x-X[i])/h),kernel)
  except:
    l=["gaussian","cauchy","epanechnikov","exp","rect","tri","biweight"]
    return f"Please choose the kernel from one of the following: \n{l}"

  return sum/(n*h)

In [10]:
# scratching integration
def simpson_integral(f,a,b,n=1000):
  x = np.linspace(a,b,n)
  y = f(x[0])+f(x[-1])

  for i in range(1,n-1):
    if i%2 == 0:
      y = y + 2*f(x[i])
    else:
      y = y + 4*f(x[i])

  return y*(b-a)/(3*n)

In [11]:
# scratching plot function
def plot_fn(X, kernel,h,a=-3,b=3,space=100):
  z = np.linspace(a,b,space)
  y = np.zeros(z.size)

  for i in range(z.size):
     y[i] = f_n(z[i],h,X,kernel)

  plt.plot(z,y, 'r-' ,lw =2)
  plt.show()

In [12]:
# creating customized plot
import plotly.graph_objects as go
def plot_function(X,h,a=-10,b=10,space=100):
  no_of_graphs=eval(input("Enter the no of Kernels you want : "))
  kernel=[input(f"Enter kernel {i+1} : ") for i in range(no_of_graphs)]
  f=input("Enter f(x) : ")
  x=np.linspace(a,b,space)
  y=[]
  for i in range(no_of_graphs):
    y.append([f_n(x[j],h,X[0,:],kernel[i]) for j in range(x.size)])

  # Create a figure
  fig = go.Figure()
  for i in range(no_of_graphs):

    fig.add_trace(go.Scatter(x=x, y=y[i], name=kernel[i]+" Kernel", visible=(i==0)))
    fig.add_trace(go.Scatter(x=x, y=den(x,f), name=f+" Density", visible=(i==0),line=dict(color="red",dash="dash")))

  buttons = []
  annotations_list = []
  MISE_values = [MISE(h,X,kernel[i],f) for i in range(no_of_graphs)]
  for i in range(no_of_graphs):
      # Set visibility for both traces of kernel[i]
      visibility = [False] * (2 * no_of_graphs)
      visibility[2 * i] = True     # Show Trace 1
      visibility[2 * i + 1] = True  # Show Trace 2
      annotations_list.append(
          dict(
              x=0.5, y=1.1,  # Position above the plot (relative coordinates)
              xref="paper", yref="paper",
              text=f"Mean Integrated Square Error for Kernel : {kernel[i]} = {MISE_values[i]}",
              showarrow=False,
              font=dict(size=12, color="black"),
          )
      )

      buttons.append(
          dict(
              label=kernel[i],
              method="update",
              args=[{"visible": visibility},
                    {"annotations": [annotations_list[i]]}],
          )
      )
  fig.update_layout(

    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=buttons,
            showactive=True,
            x=0.1,
            y=1.1,

        )
    ],
          legend=dict(
          x=0.02,
          y=0.98,
          bgcolor="rgba(255,255,255,0.7)",
          bordercolor="black",
          borderwidth=1,
          font=dict(size=12)
  ),

    annotations=[annotations_list[0]],
    title_text=f"$Plots\ of\ f_n(x)\ using\ different\ Kernels \ (h={h})$",
    title_x=0.5,
    title_font=dict(size=48, color="darkblue", family="Arial", weight="bold"),
    xaxis_title="$x$ ",
    yaxis_title="$f_n(x)$",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),



  )
  fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="linear"
    ),
    height=800,
    width=1200
  )

  fig.show()




In [13]:
# scratching MSE function
def MSE_fn(x,h,Y,kernel,density):
  z = 0

  for i in range(Y.shape[0]):
    if den(x,density) == 0:
      pass
    else:
      z += (f_n(x,h,Y[i,:],kernel) - den(x,density))**2


  return  z/Y.shape[0]

In [14]:
# scratching MISE function
def MISE(h,Y,kernel,density,a=-10,b=10,n=1000):

  kernel = kernel.lower()
  m = np.linspace(a,b,n)
  y = MSE_fn(m[0],h,Y,kernel,density)+MSE_fn(m[n-1],h,Y,kernel,density)

  for i in range(1,n-1):
    if i%2 == 0:
      y = y + 2*MSE_fn(m[i],h,Y,kernel,density)
    else:
      y = y + 4*MSE_fn(m[i],h,Y,kernel,density)

  return y*(b-a)/(3*n)

In [15]:
# function for plotting MSE
import plotly.graph_objects as go
import numpy as np

def plot_MSE(h,X,kernel,density,a=-3,b=3,space=100):
  # Create sample data
  x = np.linspace(a, b, space)
  y = [MSE_fn(x[i],h,X,kernel,density) for i in range(x.size)]

  # Create a figure
  fig = go.Figure()

  # Add a trace
  fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='MSE(x)'))

  # Update layout to change background colors
  fig.update_layout(
      title_text=f"<i>Plot of MSE using {kernel} Kernel & {density} density<i>",
      title_x=0.5,
      xaxis_title="$x$ ",
      yaxis_title="$MSE(x)$",
      title_font=dict(size=24, color="darkblue", family="Calibri", weight="bold",),
      xaxis=dict(showgrid=False),
      yaxis=dict(showgrid=False),
      height=800,
      width=1200,
  )
  fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="linear"
    )
  )

  # Show the figure
  fig.show()

In [16]:
# creating function for plots based on h
import plotly.graph_objects as go
def plot_function_h(X,kernel,f,a=-3,b=3,space=100,h=[0.1,0.3,0.5,0.7,0.9,1.1]):

  x=np.linspace(a,b,space)
  y=[]
  for i in range(len(h)):
    y.append([f_n(x[j],h[i],X[0,:],kernel) for j in range(x.size)])

  # Create a figure
  fig = go.Figure()
  for i in range(len(h)):

    fig.add_trace(go.Scatter(x=x, y=y[i], name=kernel+" Kernel", visible=(i==0)))
    fig.add_trace(go.Scatter(x=x, y=den(x,f), name=f+" Density", visible=(i==0),line=dict(color="red",dash="dash")))


  buttons = []
  annotations_list = []
  MISE_values = [MISE(h[i],X,kernel,f) for i in range(len(h))]
  for i in range(len(h)):
      # Set visibility for both the kernels and the density
      visibility = [False] * (2 * len(h))
      visibility[2 * i] = True
      visibility[2 * i + 1] = True
      annotations_list.append(
          dict(
              x=0.5, y=1.1,  # Position above the plot (relative coordinates)
              xref="paper", yref="paper",
              text=f"Mean Integrated Square Error for h : {h[i]} = {MISE_values[i]}",
              showarrow=False,
              font=dict(size=12, color="black"),
          )
      )

      buttons.append(
          dict(
              label=f"h = {h[i]}",
              method="update",
              args=[{"visible": visibility},
                    {"annotations": [annotations_list[i]]}],
          )
      )
  fig.update_layout(

        legend=dict(
            x=0.02,
            y=0.98,
            bgcolor="rgba(255,255,255,0.7)",
            bordercolor="black",
            borderwidth=1,
            font=dict(size=12)
    ),
    updatemenus=[
        dict(
            type="buttons",
            direction="left",
            buttons=buttons,
            showactive=True,
            x=0.1,
            y=1.1,

        )
    ],
    annotations=[annotations_list[0]],
    title_text=f"$Plots\ of\ f_n(x)\ using\ different\ Bandwidth \ (h={h})$",
    title_x=0.5,
    title_font=dict(size=48, color="darkblue", family="Arial", weight="bold"),
    xaxis_title="$x$ ",
    yaxis_title="$f_n(x)$",
    xaxis=dict(showgrid=False),
    yaxis=dict(showgrid=False),
    height=800,
    width=1200,



  )
  fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="linear"
    )
  )

  fig.show()


# Kernel Density Estimation
## Slide-1,2,3,4,5,6,7,8

## Slide-9
We are taking a random sample from standard normal with sample size 60. We will use 4 kernels - Gaussian,Epanechnikov,Rectangular and Exponential kernel.

In [17]:
np.random.seed(42)
n=(100,10)
m=(100,20)
sample1 = random_sample(n).cauchy()
sample2 = random_sample(m).cauchy()
sample3 = random_sample((100,30)).cauchy()
sample = random_sample((100,60)).cauchy()
normal_sample=random_sample((100,60)).gaussian()

In [18]:
plot_function(normal_sample,h=0.5)

For first 3 kernels, the estimation is almost same for any kernels. In the last case , we used exponential kernel and it resulted a slight right shift.
## Slide 10
Though for all kernels density estimation is almost similar,density estimation depends very much on smoothing parameter.


# Effects of Varying sample sizes
Here we have drawn sample from Cauchy distribution and used Gaussian, Epanechnikov, Rectangular, Triangular Kernel.
## n=10

In [19]:
plot_function(sample,h=0.5)

## n=20

In [20]:
plot_function(sample2,h=0.5)

## n=30

In [22]:
plot_function(sample3,h=0.5)

## n=60

In [ ]:
plot_function(sample,h=0.5)

Enter the no of Kernels you want : 4
Enter kernel 1 : Gaussian
Enter kernel 2 : Epanechnikov
Enter kernel 3 : Rectangular
Enter kernel 4 : Triangular
Enter f(x) : Cauchy


In [ ]:
plot_function_h(sample,"cauchy","cauchy")

Here we have plotted our density estimation for different smoothing parameter. When value of h is very small (near 0.1) there are many peaks in our estimated density function. As we increase h it gets more smoother. But for large values of h it has the problem of oversmoothing. For h=0.5, we have most similarities.

---
---
## Slide 11
Now we can illustrate crucial role of h through Buffalo annual snowfall dataset(Parzen 1979).

In [ ]:
import plotly.graph_objects as go
import numpy as np

def plot_hist(X, h, space=100):
    a=min(X)-30
    b=max(X) + 30

    no_of_graphs = eval(input("Enter the no of Kernels you want: "))
    kernel = [input(f"Enter kernel {i+1}: ") for i in range(no_of_graphs)]

    x = np.linspace(a, b, space)
    y = []
    for i in range(no_of_graphs):
        y.append([f_n(x[j], h, X, kernel[i]) for j in range(x.size)])

    # Create a figure
    fig = go.Figure()

    # Add histogram (always visible)
    fig.add_trace(
        go.Histogram(
            x=X,
            histnorm='probability density',
            name="Histogram",
            opacity=0.5,
            marker_color='lightblue',
            nbinsx=30
        )
    )

    # Add kernel density plots
    for i in range(no_of_graphs):
        fig.add_trace(
            go.Scatter(
                x=x,
                y=y[i],
                name=f"{kernel[i]} Kernel (h={h})",
                visible=(i == 0),
                line=dict(width=2)
            )
        )

    # Create buttons for kernels
    buttons = []
    for i in range(no_of_graphs):
        visibility = [True] + [False] * no_of_graphs
        visibility[i + 1] = True

        buttons.append(
            dict(
                label=kernel[i],
                method="update",
                args=[{"visible": visibility}]
            )
        )

    # Update layout
    fig.update_layout(
        updatemenus=[
            dict(
                type="buttons",
                direction="left",
                buttons=buttons,
                showactive=True,
                x=0.1,
                y=1.1,
            )
        ],
        legend=dict(
          x=0.02,
          y=0.98,
          bgcolor="rgba(255,255,255,0.7)",
          bordercolor="black",
          borderwidth=1,
          font=dict(size=12)
  ),
        title_text=f"Kernel Density Estimates (h = {h})",
        title_x=0.5,
        xaxis_title="x",
        yaxis_title="Density",
        bargap=0.01,
        template="plotly_white",
        height=800,
        width=1200,
        font=dict(color="black"),
    )
    fig.update_layout(
    xaxis=dict(
        rangeslider=dict(visible=True),
        type="linear"
    )
    )

    fig.show()

We have used Gaussian Kernel here.

In [36]:
snow = pd.read_excel("snowfall.xlsx",header=None)
snow = snow.iloc[:,0].to_numpy()

plot_hist(snow,h=6)


In [37]:
plot_hist(snow,h=12)

We used same sample data of Annual snowfall in Buffalo.<br> Now for first plot we used window length of 6 and got a trimodal distribution. Again if we use window length of 12 we get a unimodal distribution. Here the role of smoothing parameter arises. One has to use prior information about sample to choose value of smoothing parameter.
## Slide 12




In [ ]:
plot_function(sample,h=0.5)

Enter the no of Kernels you want : 4
Enter kernel 1 : Gaussian
Enter kernel 2 : Epanechnikov
Enter kernel 3 : Triangular
Enter kernel 4 : Rectangular
Enter f(x) : Cauchy


In [ ]:
plot_function(sample,h=0.9)

Enter the no of Kernels you want : 4
Enter kernel 1 : Gaussian
Enter kernel 2 : Epanechnikov
Enter kernel 3 : Rectangular
Enter kernel 4 : Triangular
Enter f(x) : Cauchy


In [ ]:
plot_function(sample,h=1.1)

Enter the no of Kernels you want : 4
Enter kernel 1 : Gaussian
Enter kernel 2 : Epanechnikov
Enter kernel 3 : Rectangular
Enter kernel 4 : Triangular
Enter f(x) : Cauchy


In [ ]:
plot_function_h(sample,"Gaussian","Cauchy",h=[0.5,0.55,0.6])

In [ ]:
plot_function_h(sample,"Rectangular","Cauchy",h=[0.8,0.9,1])

In [ ]:
plot_function_h(sample,"Triangular","Cauchy",h=[1,1.1,1.2])

In [ ]:
plot_function_h(sample,"Epanechnikov","Cauchy",h=[0.5,0.55,0.6])

## Slide 13

### Suicide Data
We have used Gaussian Kernel

In [39]:
suicide = pd.read_excel("suicide.xlsx")
suicide = suicide.iloc[:,0].to_numpy()
suicide
plot_hist(suicide,h=20)


In [40]:
plot_hist(suicide,h=60)

## Slide 14

Here we used Epanechnikov, Gaussian, Exponential, Rectangular kernels. And we have drawn sample from Uniform distribution.

In [ ]:
samples = np.random.uniform(size=(100,60))
plot_function(samples,h=0.5,a=-14)

Enter the no of Kernels you want : 4
Enter kernel 1 : Epanechnikov
Enter kernel 2 : Gaussian
Enter kernel 3 : exp
Enter kernel 4 : rect
Enter f(x) : unif


In [ ]:
sample = np.random.gumbel(size=(100,60))
plot_MSE(0.5,sample,"gaussian","gumbel_0_1")

## Probability Leakage


We used Gaussian and Epanechnikov Kernels.

In [ ]:
np.random.seed(42)
n=(100,60)
samples=random_sample(n).exp()

plot_function(samples, h=0.3,space=1000)

Enter the no of Kernels you want : 2
Enter kernel 1 : Gaussian
Enter kernel 2 : Epanechnikov
Enter f(x) : Standard Exponential


We used Epanechnikov and Rectangular kernel. Input f(x) in this format - *Chisquare_df*.

In [ ]:
sample = np.random.chisquare(df=2,size=(100,60))
plot_function(sample,h=0.5,space=1000)

Enter the no of Kernels you want : 2
Enter kernel 1 : Epanechnikov
Enter kernel 2 : Rectangular
Enter f(x) : Chisquare_2


We used Gaussian kernel here.

In [38]:
faithful = pd.read_excel("faithful.xlsx")
faithful = faithful.iloc[:,0].to_numpy()

plot_hist(faithful,h=0.25,space=1000)

In [ ]:
plot_hist(faithful,h=0.7,space=1000)

Enter the no of Kernels you want: 1
Enter kernel 1: Gaussian


In [ ]:
plot_hist(faithful,h=0.9,space=1000)

Enter the no of Kernels you want: 1
Enter kernel 1: Gaussian


Input kernel as special_den

In [ ]:
sample = np.random.uniform(low=2,high=3,size=(100,60))
plot_function(sample,h=1.7)

Enter the no of Kernels you want : 1
Enter kernel 1 : special_den
Enter f(x) : recta


## Boundary Effects


In [ ]:
sample=random_sample((100,60)).unif(low=-1,high=1)
# plot_function(sample,h=1.7,space=1000)
plot_MSE(0.5,sample,"Gaussian","Uniform_(-1)_1",space=2000)

Here the kernel is "Special" and f(x) = "Gaussian"

In [ ]:
normal_sample=random_sample((100,60)).gaussian()
plot_function(normal_sample,h=1.2,space=1000)

Enter the no of Kernels you want : 1
Enter kernel 1 : Special
Enter f(x) : Gaussian


We used Epanechnikov and Exponential kernel. If you want to input uniform(a,b) input it as Uniform_a_b. For example if you want to input Uniform(-12,-8), input Uniform_(-12)_(-8).

In [ ]:

uni_sample=random_sample((100,60)).unif(low=-12,high=-8)
plot_function(uni_sample,h=0.4,a=-15,b=-5,space=1000)

Enter the no of Kernels you want : 2
Enter kernel 1 : Epanechnikov
Enter kernel 2 : Exponential
Enter f(x) : Uniform_(-12)_(-8)


In [ ]:
plot_function(uni_sample,h=0.4,a=-15,b=-5,space=1000)

Enter the no of Kernels you want : 2
Enter kernel 1 : Epanechnikov
Enter kernel 2 : Exponential
Enter f(x) : Uniform_(-12)_(-8)


# Innovation
## Boundary Correction

In [28]:
sample_value=random_sample((100,100)).exp()
plot_function_h(sample_value,"Gaussian","Exponential",h=[0.3],space=1000)

In [25]:
a=-10
b=10
space=1000
x=np.linspace(a,b,space)
x_01=np.linspace(0,b,space)
h=0.3

kernel="Gaussian"
f="Exponential"
y=np.array([f_n(x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)])
print(len(y))
y_left=np.array([f_n(-x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)])
y+=y_left
print(f"After {len(y)}")
y1=[f_n(x[j],h,sample_value[0,:],kernel) for j in range(x.size)]
# Create a figure
fig = go.Figure()


fig.add_trace(go.Scatter(x=x, y=y1, name=kernel+" Kernel"))
fig.add_trace(go.Scatter(x=x_01, y=y, name="Reflected "+kernel+" Kernel",line=dict(color="darkgreen")))
fig.add_trace(go.Scatter(x=x, y=den(x,f), name=f+" Density",line=dict(color="red",dash="dash")))

fig.update_layout(

      legend=dict(
          x=0.02,
          y=0.98,
          bgcolor="rgba(255,255,255,0.7)",
          bordercolor="black",
          borderwidth=1,
          font=dict(size=12)
  ),

  title_text=f"$Plots\ of\ f_n(x)\ using\ Bandwidth \ (h={h})$",
  title_x=0.5,
  title_font=dict(size=48, color="darkblue", family="Arial", weight="bold"),
  xaxis_title="$x$ ",
  yaxis_title="$f_n(x)$",
  xaxis=dict(showgrid=False),  # Remove x-axis grid lines
  yaxis=dict(showgrid=False),
  height=800,
  width=1200,



)
fig.show()

1000
After 1000


See for h=0.1 in the plot below.

In [30]:
np.random.seed(42)
sample_value=random_sample((100,100)).unif()
plot_function_h(sample_value,"Gaussian","Uniform_0_1",h=[0.01,0.05,0.1,0.3],space=1000)

In [31]:
a=-3
b=3
space=1000
x=np.linspace(a,b,space)
x_01=np.linspace(0,1,space)
h=0.1

kernel="Gaussian"
f="Uniform_0_1"
y=[f_n(x[j],h,sample_value[0,:],kernel) for j in range(x.size)]
y_left=[f_n(-x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)]
y_right=[f_n(2-x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)]
# Create a figure
fig = go.Figure()


fig.add_trace(go.Scatter(x=x, y=y, name=kernel+" Kernel"))
fig.add_trace(go.Scatter(x=x, y=den(x,f), name=f+" Density",line=dict(color="red",dash="dash")))
fig.add_trace(go.Scatter(x=x_01, y=y_left, name="Left Reflection"))
fig.add_trace(go.Scatter(x=x_01, y=y_right, name="Right Reflection"))



fig.update_layout(

      legend=dict(
          x=0.02,
          y=0.98,
          bgcolor="rgba(255,255,255,0.7)",
          bordercolor="black",
          borderwidth=1,
          font=dict(size=12)
  ),

  title_text=f"$Plots\ of\ f_n(x)\ using\ Bandwidth \ (h={h})$",
  title_x=0.5,# LaTeX in title
  title_font=dict(size=48, color="darkblue", family="Arial", weight="bold"),
  xaxis_title="$x$ ",
  yaxis_title="$f_n(x)$",
  xaxis=dict(showgrid=False),
  yaxis=dict(showgrid=False),
  height=800,
  width=1200,



)


fig.show()


In [33]:
a=-3
b=3
space=1000
x=np.linspace(a,b,space)
x_01=np.linspace(0,1,space)
h=0.1

kernel="Gaussian"
f="Uniform_0_1"
y1=np.array([f_n(x[j],h,sample_value[0,:],kernel) for j in range(x.size)])
y=np.array([f_n(x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)])
y_left=np.array([f_n(-x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)])
y_right=np.array([f_n(2-x_01[j],h,sample_value[0,:],kernel) for j in range(x_01.size)])
y+=y_left+y_right
# Create a figure
fig = go.Figure()


fig.add_trace(go.Scatter(x=x, y=y1, name=kernel+" Kernel"))
fig.add_trace(go.Scatter(x=x, y=den(x,f), name=f+" Density",line=dict(color="red",dash="dash")))
fig.add_trace(go.Scatter(x=x_01, y=y, name="Final "+kernel+" Kernel",line=dict(color="darkgreen")))


fig.update_layout(


      legend=dict(
          x=0.02,
          y=0.98,
          bgcolor="rgba(255,255,255,0.7)",
          bordercolor="black",
          borderwidth=1,
          font=dict(size=12)
  ),

  title_text=f"$Plots\ of\ f_n(x)\ using\ Bandwidth \ (h={h})$",
  title_x=0.5,# LaTeX in title
  title_font=dict(size=48, color="darkblue", family="Arial", weight="bold"),
  xaxis_title="$x$ ",
  yaxis_title="$f_n(x)$",
  xaxis=dict(showgrid=False),
  yaxis=dict(showgrid=False),
  height=800,
  width=1200,



)


fig.show()
